# Sprawozdanie 11

**Grupa A3:**

inż. Michał Liss

inż. Marceli Sokólski

inż. Piotr Krzystanek

## Definicje funkcji

In [1]:
import paramiko
import os
import re
import requests
import json
import time

In [2]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def merge_results(path):
    run_in_master(f"hdfs dfs -cat {path}/part-* | hdfs dfs -put - {path}/merged.txt")
    
def get_data_from_output_path(path):
    return f"{path}/merged.txt"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}")[0]
    print("\n".join(raw[0:11]))
    
def get_time(res, max_attempts: int = 6):
    def get_id(res):
        for line in res[1]:
            m = re.search('tracking URL: http://resourcemanager:8088/proxy/(.*)/', line)
            if m != None and m.group(1) != '':
                return m.group(1)
        return ''

    def get_time_from_data(data):
        sum = 0
        for attemp in data['attempts']:
            if attemp['completed'] is False:
                # print('spark history server has not updated (yet)')
                return -1
            sum = sum + attemp['duration']
        return sum
        
    id = get_id(res)
    if id == -1:
        return -1

    attempt = 0

    while attempt < max_attempts:
        attempt = attempt + 1
        response = requests.get(f'http://namenode:18080/api/v1/applications/{id}')
        if not response.ok:
            print('WARNING: application error')
            return -1
            
        data = json.loads(response.text)
        t = get_time_from_data(data)

        if t >= 0:
            return t
        time.sleep(5)
    print('WARNING: maximum attempts exceeded')
    return -1

runs = 10

# Covid

In [3]:
measurements_covid_sql = []
measurements_covid_df = []
measurements_covid_scala = []

## SQL

In [4]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/covid/sql")
    measurements_covid_sql.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/covid_sql.py")))
    print(f"run {r} took {measurements_covid_sql[-1]}ms")
    
merge_results("/spark-result/covid/sql")
print_hdfs_output("/spark-result/covid/sql")

run 0 took 13758ms
run 1 took 14356ms
run 2 took 13586ms
run 3 took 13635ms
run 4 took 14173ms
run 5 took 13840ms
run 6 took 13625ms
run 7 took 13180ms
run 8 took 14037ms
run 9 took 13305ms
date,location,total_cases,new_cases,total_deaths,new_deaths,new_cases_per_million,average_new_cases_per_million

2022-05-03,Curacao,42035,0,273,0,0,160.00584439903545

2022-05-04,Curacao,42330,295,274,1,1543,160.00584439903545

2022-05-05,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-06,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-07,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-08,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-09,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-10,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-11,Curacao,42674,344,275,1,1799,160.00584439903545

2022-05-12,Curacao,42674,0,275,0,0,160.00584439903545



## DataFrame

In [5]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/covid/df")
    measurements_covid_df.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/covid_df.py")))
    print(f"run {r} took {measurements_covid_df[-1]}ms")
    
merge_results("/spark-result/covid/df")
print_hdfs_output("/spark-result/covid/df")

run 0 took 14344ms
run 1 took 13400ms
run 2 took 13447ms
run 3 took 14340ms
run 4 took 13724ms
run 5 took 13262ms
run 6 took 13354ms
run 7 took 14398ms
run 8 took 13666ms
run 9 took 13652ms
date,location,total_cases,new_cases,total_deaths,new_deaths,new_cases_per_million,average_new_cases_per_million

2022-05-03,Curacao,42035,0,273,0,0,160.00584439903545

2022-05-04,Curacao,42330,295,274,1,1543,160.00584439903545

2022-05-05,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-06,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-07,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-08,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-09,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-10,Curacao,42330,0,274,0,0,160.00584439903545

2022-05-11,Curacao,42674,344,275,1,1799,160.00584439903545

2022-05-12,Curacao,42674,0,275,0,0,160.00584439903545



## Scala Dataframe

In [6]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/covid01")
    measurements_covid_scala.append(get_time(run_in_master("spark-submit \
--master yarn \
--deploy-mode cluster \
--class covid01.Main \
/data/master_volume/spark_scripts/spark.jar ")))
    print(f"run {r} took {measurements_covid_scala[-1]}ms")
    
merge_results("/spark-result/covid01")
print_hdfs_output("/spark-result/covid01")

run 0 took 15406ms
run 1 took 15395ms
run 2 took 15660ms
run 3 took 16456ms
run 4 took 15502ms
run 5 took 15895ms
run 6 took 16574ms
run 7 took 16308ms
run 8 took 15907ms
run 9 took 15795ms
{"date":"2020-01-03","location":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"new_cases_per_million":0.0,"average_new_cases_per_million":160.29720723141685}

{"date":"2020-01-04","location":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"new_cases_per_million":0.0,"average_new_cases_per_million":160.29720723141685}

{"date":"2020-01-05","location":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"new_cases_per_million":0.0,"average_new_cases_per_million":160.29720723141685}

{"date":"2020-01-06","location":"Afghanistan","total_cases":0,"new_cases":0,"total_deaths":0,"new_deaths":0,"new_cases_per_million":0.0,"average_new_cases_per_million":160.29720723141685}

{"date":"2020-01-07","location":"Afghanistan","total_c

# Steam

In [7]:
measurements_steam_sql = []
measurements_steam_df = []
measurements_steam_scala = []

## SQL

In [8]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/steam/sql")
    measurements_steam_sql.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/steam_sql.py")))
    print(f"run {r} took {measurements_steam_sql[-1]}ms")
    
merge_results("/spark-result/steam/sql")
print_hdfs_output("/spark-result/steam/sql")

run 0 took 14267ms
run 1 took 14052ms
run 2 took 13415ms
run 3 took 13748ms
run 4 took 13712ms
run 5 took 13854ms
run 6 took 13764ms
run 7 took 13811ms
run 8 took 14529ms
run 9 took 13927ms
steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

907680,false,"Aug 17, 2018",907680,Wwbit,710,566,"100,000 .. 200,000",0

823550,false,"Sep 18, 2018",823550,Booty Calls,255,294,"100,000 .. 200,000",134

639780,false,"Dec 7, 2017",639780,Deep Space Waifu: FLAT JUSTICE,1449,67,"50,000 .. 100,000",4

steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

896890,false,"Dec 23, 2019",896890,VR Paradise - Steam Edition,138,50,"20,000 .. 50,000",11

726360,false,"May 22, 2020",726360,BOOBS SAGA: Prepare To Hentai Edition,367,103,"20,000 .. 50,000",0

723090,false,"Oct 24, 2017",723090,Meltys Quest,446,10,"20,000 .. 50,000",36

712790,false,"Oct 2, 2017",712790,Crimson Memories,51,21,"20,000 .. 50,000",0

825300,false,"Nov 22, 2018",825300,To Trust an Incubus,43,5,"0 .. 2

## DataFrame

In [9]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/steam/df")
    measurements_steam_df.append(get_time(run_in_master("spark-submit --master yarn --deploy-mode cluster /data/master_volume/spark_scripts/steam_df.py")))
    print(f"run {r} took {measurements_steam_df[-1]}ms")
    
merge_results("/spark-result/steam/df")
print_hdfs_output("/spark-result/steam/df")

run 0 took 13859ms
run 1 took 13817ms
run 2 took 13227ms
run 3 took 14443ms
run 4 took 14377ms
run 5 took 14525ms
run 6 took 13436ms
run 7 took 13964ms
run 8 took 13672ms
run 9 took 13799ms
steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

907680,false,"Aug 17, 2018",907680,Wwbit,710,566,"100,000 .. 200,000",0

823550,false,"Sep 18, 2018",823550,Booty Calls,255,294,"100,000 .. 200,000",134

639780,false,"Dec 7, 2017",639780,Deep Space Waifu: FLAT JUSTICE,1449,67,"50,000 .. 100,000",4

steam_appid,coming_soon,date,appid,name,positive,negative,owners,ccu

896890,false,"Dec 23, 2019",896890,VR Paradise - Steam Edition,138,50,"20,000 .. 50,000",11

726360,false,"May 22, 2020",726360,BOOBS SAGA: Prepare To Hentai Edition,367,103,"20,000 .. 50,000",0

723090,false,"Oct 24, 2017",723090,Meltys Quest,446,10,"20,000 .. 50,000",36

712790,false,"Oct 2, 2017",712790,Crimson Memories,51,21,"20,000 .. 50,000",0

825300,false,"Nov 22, 2018",825300,To Trust an Incubus,43,5,"0 .. 2

## Scala Dataframe

In [10]:
for r in range(runs):
    run_in_master(f"hdfs dfs -rm -r /spark-result/steam01")
    measurements_steam_scala.append(get_time(run_in_master("spark-submit \
--master yarn \
--deploy-mode cluster \
--class steam01.Main \
/data/master_volume/spark_scripts/spark.jar ")))
    print(f"run {r} took {measurements_steam_scala[-1]}ms")
    
merge_results("/spark-result/steam01")
print_hdfs_output("/spark-result/steam01")

run 0 took 14840ms
run 1 took 15516ms
run 2 took 16064ms
run 3 took 15516ms
run 4 took 15416ms
run 5 took 16240ms
run 6 took 15695ms
run 7 took 16246ms
run 8 took 15082ms
run 9 took 15463ms
{"key":"555310","value":{"game_id":555310,"name":"Satellite","positive":44,"negative":9,"owners":"0 .. 20,000","ccu":0,"release_date":"Feb 23, 2018"}}

{"key":"639780","value":{"game_id":639780,"name":"Deep Space Waifu: FLAT JUSTICE","positive":1449,"negative":67,"owners":"50,000 .. 100,000","ccu":4,"release_date":"Dec 7, 2017"}}

{"key":"677730","value":{"game_id":677730,"name":"Karmasutra","positive":13,"negative":6,"owners":"0 .. 20,000","ccu":0,"release_date":"Sep 29, 2017"}}

{"key":"712790","value":{"game_id":712790,"name":"Crimson Memories","positive":51,"negative":21,"owners":"20,000 .. 50,000","ccu":0,"release_date":"Oct 2, 2017"}}

{"key":"823550","value":{"game_id":823550,"name":"Booty Calls","positive":255,"negative":294,"owners":"100,000 .. 200,000","ccu":134,"release_date":"Sep 18, 201

# Wyniki

In [11]:
m_s_d = " | ".join([str(x) for x in measurements_steam_df])
m_s_s = " | ".join([str(x) for x in measurements_steam_sql])
m_c_d = " | ".join([str(x) for x in measurements_covid_df])
m_c_s = " | ".join([str(x) for x in measurements_covid_sql])

print(f"|     |  covid  |  steam  |")
print(f"|-----|---------|---------|")
print(f"| df  | {sum(measurements_covid_df) / len(measurements_covid_df)} | {sum(measurements_steam_df) / len(measurements_steam_df)} |")
print(f"| sql | {sum(measurements_covid_sql) / len(measurements_covid_sql)} | {sum(measurements_steam_sql) / len(measurements_steam_sql)} |")

print()
r = " | ".join([f"run {x}" for x in range(runs)])
print(f"|         | {r} |")
print(f"|steam df |{m_s_d}|")
print(f"|steam sql|{m_s_s}|")
print(f"|covid df |{m_c_d}|")
print(f"|covid sql|{m_c_s}|")

|     |  covid  |  steam  |
|-----|---------|---------|
| df  | 13758.7 | 13911.9 |
| sql | 13749.5 | 13907.9 |

|         | run 0 | run 1 | run 2 | run 3 | run 4 | run 5 | run 6 | run 7 | run 8 | run 9 |
|steam df |13859 | 13817 | 13227 | 14443 | 14377 | 14525 | 13436 | 13964 | 13672 | 13799|
|steam sql|14267 | 14052 | 13415 | 13748 | 13712 | 13854 | 13764 | 13811 | 14529 | 13927|
|covid df |14344 | 13400 | 13447 | 14340 | 13724 | 13262 | 13354 | 14398 | 13666 | 13652|
|covid sql|13758 | 14356 | 13586 | 13635 | 14173 | 13840 | 13625 | 13180 | 14037 | 13305|


In [12]:
measurements_steam_scala

[14840, 15516, 16064, 15516, 15416, 16240, 15695, 16246, 15082, 15463]

In [13]:
m_s_d = " | ".join([str(x) for x in measurements_steam_df])
m_s_s = " | ".join([str(x) for x in measurements_steam_sql])
m_s_scala = " | ".join([str(x) for x in measurements_steam_scala])
m_c_d = " | ".join([str(x) for x in measurements_covid_df])
m_c_s = " | ".join([str(x) for x in measurements_covid_sql])
m_c_scala =" | ".join([str(x) for x in measurements_covid_scala])

print(f"|       |  covid  |  steam  |")
print(f"|-------|---------|---------|")
print(f"| df    | {sum(measurements_covid_df) / len(measurements_covid_df)} | {sum(measurements_steam_df) / len(measurements_steam_df)} |")
print(f"| sql   | {sum(measurements_covid_sql) / len(measurements_covid_sql)} | {sum(measurements_steam_sql) / len(measurements_steam_sql)} |")
print(f"| scala | {sum(measurements_covid_scala) / len(measurements_covid_scala)} | {sum(measurements_steam_scala) / len(measurements_steam_scala)} |")

print()
r = " | ".join([f"run {x}" for x in range(runs)])
print(f"|           | {r} |")
print(f"|steam df   |{m_s_d}|")
print(f"|steam sql  |{m_s_s}|")
print(f"|steam sca  |{m_s_scala}|")
print(f"|covid df   |{m_c_d}|")
print(f"|covid sql  |{m_c_s}|")
print(f"|covid scala|{m_c_scala}|")

|       |  covid  |  steam  |
|-------|---------|---------|
| df    | 13758.7 | 13911.9 |
| sql   | 13749.5 | 13907.9 |
| scala | 15889.8 | 15607.8 |

|           | run 0 | run 1 | run 2 | run 3 | run 4 | run 5 | run 6 | run 7 | run 8 | run 9 |
|steam df   |13859 | 13817 | 13227 | 14443 | 14377 | 14525 | 13436 | 13964 | 13672 | 13799|
|steam sql  |14267 | 14052 | 13415 | 13748 | 13712 | 13854 | 13764 | 13811 | 14529 | 13927|
|steam sca  |14840 | 15516 | 16064 | 15516 | 15416 | 16240 | 15695 | 16246 | 15082 | 15463|
|covid df   |14344 | 13400 | 13447 | 14340 | 13724 | 13262 | 13354 | 14398 | 13666 | 13652|
|covid sql  |13758 | 14356 | 13586 | 13635 | 14173 | 13840 | 13625 | 13180 | 14037 | 13305|
|covid scala|15406 | 15395 | 15660 | 16456 | 15502 | 15895 | 16574 | 16308 | 15907 | 15795|
